### Semantic Distance

In [4]:
! pip install gensim
! pip install nltk

In [1]:
# import packages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import os

In [2]:
os.getcwd()

'/Users/liyutai/Desktop/MACS 30200/project'

In [3]:
ll_df = pd.read_csv("MIT_ImageRatings.csv")

In [4]:
hl_df = pd.read_csv("image_label.csv")

In [5]:
ll_df.head()

,image_name,Hue,Sat,Lum,sdHue,sdSat,sdBright,Entropy,LRSymm,UDSymm,SED,NSED,ED,Order,Natural,Habitable,Preference
0,5290333715_fbe95626ae_o.jpg,0.123053,0.280524,0.465580,0.197126,0.216424,0.275851,7.434824,0.005822,0.004986,0.121669,0.008589,0.096019,1.00,2.17,2.93,1.53
1,4357587050_d09069f341_o.jpg,0.276483,0.254353,0.566156,0.278846,0.294493,0.270801,6.177281,0.008411,0.007431,0.077890,0.034188,0.100465,1.06,2.00,1.59,1.74
2,sun_audijvtzzhgkrybg.jpg,0.278962,0.202144,0.497832,0.259493,0.157821,0.232531,7.738311,0.010022,0.008939,0.059931,0.102152,0.134441,1.07,2.04,2.10,1.95
3,sun_ajjodogkvthozlni.jpg,0.378697,0.205623,0.475785,0.220284,0.194211,0.314798,7.733583,0.008954,0.008261,0.086321,0.079031,0.123857,1.09,1.16,2.08,1.26
4,sun_dgqyeayssxdxcaim.jpg,0.389366,0.175528,0.538711,0.344595,0.135950,0.194159,7.118987,0.009003,0.003885,0.040296,0.077255,0.111384,1.12,1.85,2.81,1.61


In [6]:
hl_df.head()

,image_name,labels
0,2755860336_126b78842a_o.jpg,"['Plant(96.71%)', 'Building(96.41%)', 'Window(..."
1,5005678830_2e45739d41_o.jpg,"['Plant(97.29%)', 'Sky(96.57%)', 'Building(96...."
2,sun_bggixrgemhvuewkc.jpg,"['Plant(96.76%)', 'Building(96.48%)', 'Car(95...."
3,sun_arkbrhiinafdbhrw.jpg,"['Plant(96.95%)', 'Plant community(93.29%)', '..."
4,sun_blplndgggpunourm.jpg,"['Water(98.00%)', 'Plant(96.93%)', 'Water reso..."


In [7]:
dataframe = ll_df.join(hl_df.set_index('image_name'), on='image_name')

In [8]:
dataframe = dataframe[['image_name', 'Natural', 'labels']]

In [9]:
dataframe.head()

,image_name,Natural,labels
0,5290333715_fbe95626ae_o.jpg,2.17,"['Building(94.69%)', 'Daytime(94.46%)', 'Prope..."
1,4357587050_d09069f341_o.jpg,2.00,"['Building(95.83%)', 'Window(94.71%)', 'Proper..."
2,sun_audijvtzzhgkrybg.jpg,2.04,"['Daytime(94.42%)', 'Property(94.33%)', 'Sky(9..."
3,sun_ajjodogkvthozlni.jpg,1.16,"['Sky(95.29%)', 'Building(92.38%)', 'Urban des..."
4,sun_dgqyeayssxdxcaim.jpg,1.85,"['Sky(95.24%)', 'Cloud(93.56%)', 'Wood(85.84%)..."


In [10]:
dataframe.isnull().sum()

image_name     0
Natural        0
labels        75
dtype: int64

In [11]:
miss_val = pd.isnull(dataframe['labels'])

In [12]:
dataframe[miss_val]

,image_name,Natural,labels
214,sun_abnxhmmesrvgqazg.jpg,2.61,NaN
224,sun_aglpxonwfoqtmeub.jpg,6.79,NaN
239,sun_bbpefokgivhwzglh.jpg,1.58,NaN
272,sun_bccjpnbmxhyfptmc.jpg,2.60,NaN
274,sun_bbzwbfgqadhzqhqf.jpg,6.00,NaN
...,...,...,...
1076,sun_byyoeixlskdvtnxu.jpg,1.70,NaN
1081,sun_badbdepxqipklvvf.jpg,6.93,NaN
1085,labelme_cwyzvggchetltzv.jpg,1.71,NaN
1097,sun_aaqeqhpbmudvvcdj.jpg,1.50,NaN


In [13]:
clean_df = dataframe[pd.notnull(dataframe['labels'])]

In [14]:
len(clean_df)

1030

In [15]:
clean_df[clean_df['labels'] == "[]"]

,image_name,Natural,labels
199,4463022243_8881fc7bda_o.jpg,1.47,[]
259,sun_bvnbbmftbvgjxxnf.jpg,6.42,[]
655,sun_bewlahslhahuvzjt.jpg,6.90,[]
960,sun_bdtmmtsvfiwkcdfh.jpg,1.12,[]
1059,sun_bkdzqjanktzjkvjb.jpg,4.28,[]
1100,sun_alewbcnalibklcwh.jpg,2.08,[]


In [16]:
clean_df2 = clean_df[clean_df['labels'] != "[]"]

In [17]:
clean_df2.head()

,image_name,Natural,labels
0,5290333715_fbe95626ae_o.jpg,2.17,"['Building(94.69%)', 'Daytime(94.46%)', 'Prope..."
1,4357587050_d09069f341_o.jpg,2.00,"['Building(95.83%)', 'Window(94.71%)', 'Proper..."
2,sun_audijvtzzhgkrybg.jpg,2.04,"['Daytime(94.42%)', 'Property(94.33%)', 'Sky(9..."
3,sun_ajjodogkvthozlni.jpg,1.16,"['Sky(95.29%)', 'Building(92.38%)', 'Urban des..."
4,sun_dgqyeayssxdxcaim.jpg,1.85,"['Sky(95.24%)', 'Cloud(93.56%)', 'Wood(85.84%)..."


In [18]:
len(clean_df2.loc[0]['labels'].split(","))

10

In [42]:
dataframe = clean_df2[:]
for x in range(len(dataframe)):
    dataframe['num_label'] = len(dataframe.iloc[x]['labels'].split(","))

<ipython-input-42-28bb8b0ee409>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['num_label'] = len(dataframe.iloc[x]['labels'].split(","))


In [52]:
dataframe['num_label'].value_counts()

10    1024
Name: num_label, dtype: int64